# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import re
import pickle

#import nltk stuff
import nltk

nltk.download(['punkt','stopwords', 'wordnet', 'punkt_tab'])

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

#import sklearn stuff
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\P334257\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\P334257\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\P334257\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\P334257\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///data/DisasterResponse.db')
df = pd.read_sql('SELECT * FROM TAB_DR', engine)
X = df.message
y = df.drop(columns=['id', 'message', 'original', 'genre']) 

#check shapes
print(f"Shape of X: {X.shape} // Shape of Y: {y.shape}")
df.head()

Shape of X: (26028,) // Shape of Y: (26028, 36)


,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1.0,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1.0,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
#check data quality: are there only 0 and 1 ?!
condition = (y > 1) | (y < 0) | (y == np.nan)
rows = condition.any(axis=1)
rows.sum()

np.int64(0)

In [4]:
index = df[rows].index
index

Index([], dtype='int64')

### 2. Write a tokenization function to process your text data

In [10]:
def tokenize(text):
    # Replace all URLs
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    
    # Normalize
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)

    # Tokenize Text
    tokens = word_tokenize(text)

    # Remove Stopwords
    words = [t for t in tokens if t not in stopwords.words("english")]

    # Lemmatize
    clean_tokens = []
    for word in words:
        clean_tok = WordNetLemmatizer().lemmatize(word).lower().strip()
        clean_tokens.append(clean_tok)
    
    return clean_tokens

<>:3: SyntaxWarning: invalid escape sequence '\('
<>:3: SyntaxWarning: invalid escape sequence '\('
C:\Users\P334257\AppData\Local\Temp\1\ipykernel_18696\2633273962.py:3: SyntaxWarning: invalid escape sequence '\('
  url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'


In [11]:
#Test tokenize function
text = "The first time you see The Second Renaissance it may look boring. Look at it at least twice and definitely watch part 2. It will change your view of the matrix. Are the human people the ones who started the war ? Is AI a bad thing ?"
text = tokenize(text)
print(text)

['the', 'first', 'time', 'see', 'the', 'second', 'renaissance', 'may', 'look', 'boring', 'look', 'least', 'twice', 'definitely', 'watch', 'part', '2', 'it', 'change', 'view', 'matrix', 'are', 'human', 'people', 'one', 'started', 'war', 'is', 'ai', 'bad', 'thing']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [18]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=10), n_jobs=4))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [19]:
#Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [20]:
#Train pipeline
pipeline.fit(X_train.values, y_train)

c:\Users\P334257\Documents\Udacity\04_Data_Science\03_Data_Engineering\10_Project\.venv\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x000001B67FDF9D00>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators=10),
                                       n_jobs=4))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [21]:
# make the predictions on the test data
y_pred = pipeline.predict(X_test)

In [22]:
# Check for shape consistency
print(f"y_pred Shape: {y_pred.shape}")
print(f"y_test Shape: {y_test.shape}")

y_pred Shape: (6507, 36)
y_test Shape: (6507, 36)


In [23]:
print(classification_report(y_test.values,
                            y_pred,
                            target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.84      0.92      0.88      4938
               request       0.78      0.43      0.56      1104
                 offer       0.00      0.00      0.00        26
           aid_related       0.75      0.62      0.68      2714
          medical_help       0.64      0.09      0.16       542
      medical_products       0.70      0.06      0.11       347
     search_and_rescue       0.73      0.11      0.20       192
              security       0.20      0.01      0.01       133
              military       0.59      0.08      0.15       225
           child_alone       0.00      0.00      0.00         0
                 water       0.85      0.35      0.50       430
                  food       0.80      0.51      0.62       731
               shelter       0.76      0.24      0.36       598
              clothing       0.85      0.17      0.28       101
                 money       0.88      

c:\Users\P334257\Documents\Udacity\04_Data_Science\03_Data_Engineering\10_Project\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\P334257\Documents\Udacity\04_Data_Science\03_Data_Engineering\10_Project\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\P334257\Documents\Udacity\04_Data_Science\03_Data_Engineering\10_Project\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor 

### 6. Improve your model
Use grid search to find better parameters. 

In [24]:
#get the pipeline params
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x000001B67FDF9D00>)),
  ('tfidf', TfidfTransformer()),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators=10),
                         n_jobs=4))],
 'transform_input': None,
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x000001B67FDF9D00>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators=10),
                       n_jobs=4),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main_

In [27]:
parameters = {
    'clf__estimator__n_estimators': [10, 25],
    'clf__estimator__min_samples_leaf': [1, 2]
}

cv = GridSearchCV(pipeline, parameters, verbose=3, cv=2)

cv.fit(X_train, y_train)

Fitting 2 folds for each of 4 candidates, totalling 8 fits


c:\Users\P334257\Documents\Udacity\04_Data_Science\03_Data_Engineering\10_Project\.venv\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV 1/2] END clf__estimator__min_samples_leaf=1, clf__estimator__n_estimators=10;, score=0.230 total time= 2.7min


c:\Users\P334257\Documents\Udacity\04_Data_Science\03_Data_Engineering\10_Project\.venv\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV 2/2] END clf__estimator__min_samples_leaf=1, clf__estimator__n_estimators=10;, score=0.238 total time= 2.7min


...

[CV 1/2] END clf__estimator__min_samples_leaf=2, clf__estimator__n_estimators=25;, score=0.222 total time= 4.1min


c:\Users\P334257\Documents\Udacity\04_Data_Science\03_Data_Engineering\10_Project\.venv\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV 2/2] END clf__estimator__min_samples_leaf=2, clf__estimator__n_estimators=25;, score=0.233 total time= 5.2min


c:\Users\P334257\Documents\Udacity\04_Data_Science\03_Data_Engineering\10_Project\.venv\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


GridSearchCV(cv=2,
             estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x000001B67FDF9D00>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators=10),
                                                              n_jobs=4))]),
             param_grid={'clf__estimator__min_samples_leaf': [1, 2],
                         'clf__estimator__n_estimators': [10, 25]},
             verbose=3)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [28]:
y_pred_cv = cv.predict(X_test)
print(classification_report(y_test.values, 
                            y_pred_cv, 
                            target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.83      0.95      0.89      4938
               request       0.77      0.51      0.62      1104
                 offer       0.00      0.00      0.00        26
           aid_related       0.76      0.69      0.72      2714
          medical_help       0.63      0.11      0.19       542
      medical_products       0.82      0.18      0.30       347
     search_and_rescue       0.62      0.03      0.05       192
              security       0.33      0.01      0.01       133
              military       0.67      0.10      0.17       225
           child_alone       0.00      0.00      0.00         0
                 water       0.86      0.38      0.53       430
                  food       0.81      0.65      0.72       731
               shelter       0.82      0.36      0.50       598
              clothing       0.71      0.15      0.25       101
                 money       0.78      

c:\Users\P334257\Documents\Udacity\04_Data_Science\03_Data_Engineering\10_Project\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\P334257\Documents\Udacity\04_Data_Science\03_Data_Engineering\10_Project\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\P334257\Documents\Udacity\04_Data_Science\03_Data_Engineering\10_Project\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor 

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [29]:
# Build Pipe 2
pipeline_2 = Pipeline([
    
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(AdaBoostClassifier(learning_rate=0.1, 
                                                         n_estimators=10)
                                      ,n_jobs=10))
   
])

In [30]:
# Train Pipe 2
pipeline_2.fit(X_train, y_train)

c:\Users\P334257\Documents\Udacity\04_Data_Science\03_Data_Engineering\10_Project\.venv\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x000001B67FDF9D00>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=AdaBoostClassifier(learning_rate=0.1,
                                                                    n_estimators=10),
                                       n_jobs=10))])

In [31]:
# Test Pipe 2
y_pred_2 = pipeline_2.predict(X_test)

# Classification Report Pipe 2
print(classification_report(y_test.values, 
                            y_pred_2, 
                            target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.76      1.00      0.86      4938
               request       0.82      0.17      0.28      1104
                 offer       0.00      0.00      0.00        26
           aid_related       0.80      0.20      0.32      2714
          medical_help       0.70      0.03      0.06       542
      medical_products       1.00      0.01      0.01       347
     search_and_rescue       0.77      0.17      0.28       192
              security       0.00      0.00      0.00       133
              military       0.00      0.00      0.00       225
           child_alone       0.00      0.00      0.00         0
                 water       0.58      0.87      0.70       430
                  food       0.76      0.70      0.73       731
               shelter       0.88      0.31      0.46       598
              clothing       0.77      0.10      0.18       101
                 money       1.00      

c:\Users\P334257\Documents\Udacity\04_Data_Science\03_Data_Engineering\10_Project\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\P334257\Documents\Udacity\04_Data_Science\03_Data_Engineering\10_Project\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\P334257\Documents\Udacity\04_Data_Science\03_Data_Engineering\10_Project\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor 

In [32]:
# Get Params of Pipe 2
pipeline_2.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x000001B67FDF9D00>)),
  ('tfidf', TfidfTransformer()),
  ('clf',
   MultiOutputClassifier(estimator=AdaBoostClassifier(learning_rate=0.1,
                                                      n_estimators=10),
                         n_jobs=10))],
 'transform_input': None,
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x000001B67FDF9D00>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=AdaBoostClassifier(learning_rate=0.1,
                                                    n_estimators=10),
                       n_jobs=10),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 

In [33]:
# Run a GridSearch on Pipe 2
parameters_2 = {
    'clf__estimator__n_estimators': [10, 50, 100],
    'clf__estimator__learning_rate': [0.1, 0.4, 0.7]
}

cv_2 = GridSearchCV(pipeline_2, parameters_2, verbose=10, cv=2)

cv_2.fit(X_train, y_train)

Fitting 2 folds for each of 9 candidates, totalling 18 fits
[CV 1/2; 1/9] START clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=10


c:\Users\P334257\Documents\Udacity\04_Data_Science\03_Data_Engineering\10_Project\.venv\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV 1/2; 1/9] END clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=10;, score=0.194 total time= 2.6min
[CV 2/2; 1/9] START clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=10


c:\Users\P334257\Documents\Udacity\04_Data_Science\03_Data_Engineering\10_Project\.venv\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV 2/2; 1/9] END clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=10;, score=0.199 total time= 2.6min
[CV 1/2; 2/9] START clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=50


...

[CV 1/2; 9/9] END clf__estimator__learning_rate=0.7, clf__estimator__n_estimators=100;, score=0.204 total time= 2.9min
[CV 2/2; 9/9] START clf__estimator__learning_rate=0.7, clf__estimator__n_estimators=100


c:\Users\P334257\Documents\Udacity\04_Data_Science\03_Data_Engineering\10_Project\.venv\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV 2/2; 9/9] END clf__estimator__learning_rate=0.7, clf__estimator__n_estimators=100;, score=0.203 total time= 2.8min


c:\Users\P334257\Documents\Udacity\04_Data_Science\03_Data_Engineering\10_Project\.venv\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


GridSearchCV(cv=2,
             estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x000001B67FDF9D00>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=AdaBoostClassifier(learning_rate=0.1,
                                                                                           n_estimators=10),
                                                              n_jobs=10))]),
             param_grid={'clf__estimator__learning_rate': [0.1, 0.4, 0.7],
                         'clf__estimator__n_estimators': [10, 50, 100]},
             verbose=10)

In [34]:
# Test Best Grid Search result Pipe 2
y_pred_2_cv = cv_2.predict(X_test)

# Classification Report Pipe 2
print(classification_report(y_test.values, y_pred_2_cv, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.76      1.00      0.86      4938
               request       0.83      0.25      0.39      1104
                 offer       0.00      0.00      0.00        26
           aid_related       0.80      0.27      0.40      2714
          medical_help       0.64      0.03      0.06       542
      medical_products       0.76      0.05      0.09       347
     search_and_rescue       1.00      0.02      0.03       192
              security       0.00      0.00      0.00       133
              military       0.29      0.01      0.02       225
           child_alone       0.00      0.00      0.00         0
                 water       0.84      0.49      0.62       430
                  food       0.76      0.70      0.73       731
               shelter       0.87      0.33      0.48       598
              clothing       0.79      0.11      0.19       101
                 money       0.88      

c:\Users\P334257\Documents\Udacity\04_Data_Science\03_Data_Engineering\10_Project\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\P334257\Documents\Udacity\04_Data_Science\03_Data_Engineering\10_Project\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\P334257\Documents\Udacity\04_Data_Science\03_Data_Engineering\10_Project\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor 

### 9. Export your model as a pickle file

In [36]:
pickle.dump(cv_2, open('models/classifier.pkl', 'wb') )

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.